# NOAA RATPAC-B Data

-----

## Initial Data Exploration

Initial data exploration for the NOAA RATPAC-B temperature data.

In [1]:
# Imports
import os
import pickle
import numpy as np
import pandas as pd
import calendar
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
# Load data
proc_data_dir = '../data/processed'

with open(os.path.join(proc_data_dir, 'ratpac_stations.pkl'), 'rb') as fin:
    station_data = pickle.load(fin)

with open(os.path.join(proc_data_dir, 'ratpac_b.pkl'), 'rb') as fin:
    ratpac_data = pickle.load(fin)

# - Merge to add location data and dates
all_data = ratpac_data.join(station_data.loc[:, ['LAT', 'LON']], on='station_number')

dt_data = np.empty((all_data.shape[0], 1), dtype=datetime)
for row_id, row_vals in enumerate(all_data.values):
    dt_data[row_id] = datetime(row_vals[1], row_vals[2], 
                               calendar.monthrange(row_vals[1], row_vals[2])[1])
all_data['Date'] = dt_data

all_data.drop(['year', 'month', 'station_number', 'station_id'], axis=1, inplace=True)